### Setup

In [1]:
import datetime

import os
pwd = os.path.abspath('.')
os.chdir(os.path.join(pwd, '../src/'))

%load_ext autoreload
%autoreload 2

  ### Get Index data for date range

In [2]:
from markets_insights.datareader import data_reader
import datetime

reader = data_reader.NseIndicesReader()

from_date = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = reader.read(data_reader.DateRangeCriteria(from_date, to_date))
print(result.head(3).to_markdown())

|    | Identifier    | Index Date   |    Open |    High |     Low |   Close |   Points Change |   Change(%) |      Volume |   Turnover |   P/E |   P/B |   Div Yield | Date                |   PreviousClose |
|---:|:--------------|:-------------|--------:|--------:|--------:|--------:|----------------:|------------:|------------:|-----------:|------:|------:|------------:|:--------------------|----------------:|
|  0 | Nifty 50      | 02-01-2023   | 18131.7 | 18215.2 | 18086.5 | 18197.5 |           92.15 |        0.51 | 2.56074e+08 |        nan | 21.9  |  4.27 |        1.26 | 2023-01-02 00:00:00 |             nan |
|  1 | Nifty Next 50 | 02-01-2023   | 42321.1 | 42409.8 | 42117.7 | 42248.4 |           60.75 |        0.14 | 1.34929e+08 |        nan | 25.67 |  4.79 |        1.76 | 2023-01-02 00:00:00 |             nan |
|  2 | Nifty 100     | 02-01-2023   | 18290.2 | 18350.9 | 18237.5 | 18334.3 |           75.55 |        0.41 | 3.91003e+08 |        nan | 22.75 |  4.41 |        1.3  | 2023-

### Get daily, monthly and annually aggregrated data
In this example we will use HistoricalDataProcessor class to get data between a date range. HistoricalDataProcessor will also do monthly and annual aggregation of data.

In [22]:
# import classes & setup
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor
from markets_insights.datareader.data_reader import NseIndicesReader, DateRangeCriteria
histDataProcessor = HistoricalDataProcessor()

# Fetch and process the data
year_start = datetime.date(2023, 1, 1)
year_end = datetime.date(2023, 12, 31)
result = histDataProcessor.process(NseIndicesReader(), DateRangeCriteria(year_start, year_end))

Started to read data
Reading data from 2023-01-02 to 2023-12-29
HistoricalDataProcessor.get_data took 12 seconds
Started periodic calculation for Month
HistoricalDataProcessor.add_monthly_growth_calc took 0 seconds
Started periodic calculation for Year
HistoricalDataProcessor.add_yearly_growth_calc took 0 seconds
HistoricalDataProcessor.process took 12 seconds


### Calculation pipeline for RSI

In [23]:
import datetime
from markets_insights.datareader.data_reader import BhavCopyReader, DateRangeCriteria
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder, HistoricalDataProcessOptions

reader = BhavCopyReader()
options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(reader, DateRangeCriteria(year_start, to_date))

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline
histDataProcessor.run_calculation_pipelines()

Started to read data
Reading data from 2023-01-02 to 2023-12-29
HistoricalDataProcessor.get_data took 69 seconds
HistoricalDataProcessor.process took 69 seconds


c:\Data\_Personal\Projects\markets_insights\src\markets_insights\calculations\base.py:386: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = data.groupby(self.get_group_cols(data.columns), group_keys=True).apply(


RsiCalculationWorker took 8 seconds
ColumnValueCrossedAboveFlagWorker took 1 seconds
ColumnValueCrossedBelowFlagWorker took 1 seconds


In [24]:
from markets_insights.core.column_definition import BaseColumns, CalculatedColumns

print(result.get_daily_data().sort_values(BaseColumns.Date).tail(3)[[BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, CalculatedColumns.RelativeStrengthIndex]].to_markdown())

|        | Identifier   | Date                |   Close |     Rsi |
|-------:|:-------------|:--------------------|--------:|--------:|
| 153931 | HDFCLIQUID   | 2023-12-29 00:00:00 | 1000    | 50.9716 |
| 408090 | TOKYOPLAST   | 2023-12-29 00:00:00 |  118.25 | 51.86   |
| 445185 | ZYDUSWELL    | 2023-12-29 00:00:00 | 1681.1  | 72.0317 |


### Test Calculation

In [30]:
from_date = datetime.date(2024, 1, 1)
to_date = datetime.date(2024, 1, 31)
reader.read(DateRangeCriteria(from_date, to_date))

,Nifty 50-Identifier,Nifty 50-Index Date,Nifty 50-Open,Nifty 50-High,Nifty 50-Low,Nifty 50-Close,Nifty 50-Points Change,Nifty 50-Change(%),Nifty 50-Volume,Nifty 50-Turnover,...,India VIX-Turnover,India VIX-P/E,India VIX-P/B,India VIX-Div Yield,India VIX-PreviousClose,Identifier,Close,High,Low,Open
0,Nifty 50,01-01-2024,21727.75,21834.35,21680.85,21741.90,10.50,.05,153995217.0,1.418409e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1481.055858,1468.595931,1513.233293,1498.207206
1,Nifty 50,02-01-2024,21751.35,21755.60,21555.65,21665.80,-76.10,-.35,263711568.0,2.531613e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1485.994513,1442.679045,1513.473758,1481.699591
2,Nifty 50,03-01-2024,21661.10,21677.00,21500.35,21517.35,-148.45,-.69,311933117.0,3.232923e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1526.053191,1466.644114,1533.275094,1485.672154
3,Nifty 50,04-01-2024,21605.80,21685.65,21564.55,21658.60,141.25,.66,339172697.0,3.086815e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1624.801200,1526.352279,1632.132450,1532.597978
4,Nifty 50,05-01-2024,21705.75,21749.60,21629.20,21710.80,52.20,.24,309303266.0,2.655449e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1718.986540,1614.969371,1732.068068,1628.338335
5,Nifty 50,08-01-2024,21747.60,21763.95,21492.90,21513.00,-197.80,-.91,231452935.0,2.114094e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1598.291233,1600.879000,1701.733967,1721.900238
6,Nifty 50,09-01-2024,21653.60,21724.45,21517.85,21544.85,31.85,.15,228568589.0,2.405574e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1624.800151,1608.622732,1684.371820,1608.438254
7,Nifty 50,10-01-2024,21529.30,21641.85,21448.65,21618.70,73.85,.34,216991926.0,2.095896e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1666.823439,1486.646059,1674.367681,1623.321395
8,Nifty 50,11-01-2024,21688.00,21726.50,21593.75,21647.20,28.50,.13,212453866.0,2.349212e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1695.160532,1647.194845,1709.718923,1672.166538
9,Nifty 50,12-01-2024,21773.55,21928.25,21715.15,21894.55,247.35,1.14,294678459.0,3.056550e+11,...,NaN,-,-,-,NaN,Nifty 50 / India VIX,1671.339695,1650.602183,1748.401771,1705.388682


In [40]:
result.get_daily_data()['Date'].min()

Timestamp('1991-12-02 00:00:00')

In [38]:
# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline
histDataProcessor.run_calculation_pipelines()
result.get_daily_data()

RsiCalculationWorker took 0 seconds
ColumnValueCrossedAboveFlagWorker took 0 seconds
ColumnValueCrossedBelowFlagWorker took 0 seconds


,Date,Open,High,Low,Close,Identifier,PreviousClose,Volume,Turnover,Month,...,Year,Year Open,Year Close,Year Low,Year High,Year Volume,Year Turnover,Rsi,RsiCrossedAbove,RsiCrossedBelow
0,1991-12-02,563.45,563.45,563.45,563.45,NIFTY 50,561.06,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
1,1991-12-03,561.06,561.06,561.06,561.06,NIFTY 50,559.31,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
2,1991-12-04,559.31,559.31,559.31,559.31,NIFTY 50,555.34,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
3,1991-12-05,555.34,555.34,555.34,555.34,NIFTY 50,561.97,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
4,1991-12-06,561.97,561.97,561.97,561.97,NIFTY 50,554.89,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
5,1991-12-09,554.89,554.89,554.89,554.89,NIFTY 50,550.49,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
6,1991-12-10,550.49,550.49,550.49,550.49,NIFTY 50,547.26,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
7,1991-12-11,547.26,547.26,547.26,547.26,NIFTY 50,540.64,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
8,1991-12-12,540.64,540.64,540.64,540.64,NIFTY 50,537.64,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False
9,1991-12-16,537.64,537.64,537.64,537.64,NIFTY 50,533.57,0,0,1991-12,...,1991,563.45,558.63,533.57,563.45,0,0,NaN,False,False


In [18]:
from markets_insights.core.environment import EnvironmentSettings

EnvironmentSettings.Paths

{'DataBaseDir': '../../_data',
 'RawDataDir': 'raw',
 'ProcessedDataDir': 'processed',
 'HistoricalDataDir': 'historical',
 'MonthlySuffix': 'monthly',
 'AnnualSuffix': 'annual',
 'BhavDataDir': 'bhavcopy',
 'NseDerivativesDataDir': 'nse_derivatives',
 'NseIndicesDataDir': 'nse_indices',
 'ManualDataPath': '../manual_data'}

In [8]:
daily_data[daily_data[workers[1].get_columns()[0]] == False][['Identifier', 'Date', 'Sma50', workers[1].get_columns()[0]]]

,Identifier,Date,Sma50,CloseAboveSma50
0,OSWALAGRO,2023-10-03,NaN,False
1,ORIENTPPR,2023-10-03,NaN,False
2,PAKKA,2023-10-03,NaN,False
3,PAISALO,2023-10-03,NaN,False
4,ORIENTHOT,2023-10-03,NaN,False
...,...,...,...,...
109019,YATHARTH,2023-12-29,382.476,False
109026,ZEEMEDIA,2023-12-29,15.184,False
109029,ZIMLAB,2023-12-29,116.195,False
109030,ZODIACLOTH,2023-12-29,132.779,False


In [15]:
data[['Identifier', 'Date', workers[1].get_columns()[0]]].tail(10)

,Identifier,Date,CloseCrossedBelowSma50
446701,STERTOOLS,2023-12-28,True
446743,SHAKTIPUMP,2023-12-28,True
446790,AAVAS,2023-12-28,True
446810,ABFRL,2023-12-28,True
446814,AGARIND,2023-12-28,True
446835,AMBER,2023-12-28,True
446866,BALKRISIND,2023-12-28,True
446915,CMSINFO,2023-12-28,True
446956,ASTRAL,2023-12-28,True
446980,ABSLLIQUID,2023-12-28,True


In [63]:
from markets_insights.core.core import IdentifierFilter, InstrumentTypeFilter
from markets_insights.datareader.data_reader import BhavCopyReader, NseDerivatiesReader


from datetime import date

cash_reader = BhavCopyReader().set_filter(IdentifierFilter("INDIACEM"))
filters = IdentifierFilter("INDIACEM") & InstrumentTypeFilter('FUTSTK')
fut_reader = NseDerivatiesReader().set_filter(filters)

premium_reader = cash_reader - fut_reader

#filters.get_query()
fut_reader.read(date(2024, 2, 21))

,InstrumentType,Identifier,ExpiryDate,StrkPric,OptionType,PreviousClose,Open,High,Low,Close,...,LftmLw,UnitOfMeasr,TtlNbOfTxsExctd,Ccy,Rsvd01,Rsvd02,Rsvd03,Rsvd04,Rsvd05,Date
0,FUTSTK,INDIACEM,28-Mar-2024,0.0,XX,243.20,246.85,246.85,243.55,243.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-21
1,FUTSTK,INDIACEM,29-Feb-2024,0.0,XX,242.45,245.00,247.60,240.10,241.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-21
2,FUTSTK,INDIACEM,25-Apr-2024,0.0,XX,251.50,0.00,0.00,0.00,251.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-21


In [36]:
from markets_insights.core.column_definition import BaseColumns
import pandas as pd
import pandas_ta as pd_ta

data = pd.DataFrame(daily_data.tail(100))
data.index = data[BaseColumns.Date]
pd_ta.vwap(
  close=data[BaseColumns.Close],
  high=data[BaseColumns.High],
  low=data[BaseColumns.Low],
  volume=data[BaseColumns.Volume],
)

Date
2024-01-01     611.533333
2024-01-02     612.100000
2024-01-03     607.966667
2024-01-04     609.716667
2024-01-05     608.516667
2024-01-08     603.566667
2024-01-09     620.850000
2024-01-10     607.750000
2024-01-11     606.133333
2024-01-12     603.416667
2024-01-15     598.300000
2024-01-16     589.366667
2024-01-17     581.783333
2024-01-18     583.216667
2024-01-19     582.133333
2024-01-23     575.683333
2024-01-24     580.250000
2024-01-25     578.316667
2024-01-29     575.516667
2024-01-30     579.866667
2024-01-31     597.750000
2024-02-01     610.466667
2024-02-02     615.850000
2024-02-05     615.866667
2024-02-06     617.033333
2024-02-07     615.900000
2024-02-08     613.950000
2024-02-09     604.583333
2024-02-12     678.100000
2024-02-13     732.600000
2024-02-14     819.433333
2024-02-15     919.433333
2024-02-16     930.283333
2024-02-19     981.083333
2024-02-20    1007.583333
Name: VWAP_D, dtype: float64

In [16]:
from markets_insights.core.column_definition import BaseColumns
daily_data = result.get_daily_data()

if len(daily_data[BaseColumns.Identifier].unique()) > 1:
    data = daily_data.groupby(BaseColumns.Identifier)
    data.apply(
        lambda x: x[BaseColumns.Turnover].rolling(50).sum()
        / x.rolling(50)[BaseColumns.Volume].sum()
    ) \
    .reset_index(level=0, drop=True)
else:
    daily_data[BaseColumns.Turnover].rolling(50).sum() / daily_data[BaseColumns.Volume].rolling(50).sum()

In [17]:
daily_data[BaseColumns.Turnover].rolling(50).sum() / daily_data[BaseColumns.Volume].rolling(50).sum()

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
          ...     
241    2390.315815
242    2393.762796
243    2397.837564
244    2403.204247
245    2407.309569
Length: 246, dtype: float64

### A real use case: Understand the affect of RSI and Stochastic RSI on price
In this use case, understand the affect of RSI and Stochastic RSI on price

#### Preparing the data
We perform below steps to prepare our analysis data
- Calculate RSI for each day for all the stocks.
- Add a flag for whenever the RSI crosses the control limits (eg: above 75 and below 30)
- Calculate the highest and lowest price change in the next 1, 3, 5, 7 & 10 trading sessions.
- Find the median for highest price change and lowest price change whenever the RSI crosses the control limits.

In [7]:
# import classes
import datetime
from markets_insights.datareader import data_reader
from markets_insights.dataprocess import data_processor

# Fetch the data
reader = data_reader.BhavCopyReader()
options = data_processor.HistoricalDataProcessOptions(include_monthly_data = False, include_annual_data=False)
histDataProcessor = data_processor.HistoricalDataProcessor(options)

from_date = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(data_reader.BhavCopyReader(), data_reader.DateRangeCriteria(from_date, to_date))

Started to read data
Reading data from 2023-01-02 to 2023-12-29
HistoricalDataProcessor.get_data took 68 seconds
HistoricalDataProcessor.process took 68 seconds


In [13]:
# prepare calculation pipeline
periods = [1, 7, 15, 30, 45]

pipelines = data_processor.MultiDataCalculationPipelines()
pipelines.set_item('rsi', data_processor.CalculationPipelineBuilder.create_rsi_calculation_pipeline(crossing_above_flag_value = 75, crossing_below_flag_value = 30, window = 14))
pipelines.set_item('foward_looking_fall', data_processor.CalculationPipelineBuilder.create_forward_looking_price_fall_pipeline(periods))
pipelines.set_item('foward_looking_rise', data_processor.CalculationPipelineBuilder.create_forward_looking_price_rise_pipeline(periods))
histDataProcessor.set_calculation_pipelines(pipelines=pipelines)

# run the pipeline and show results
histDataProcessor.run_calculation_pipelines()

daily_data = result.get_daily_data()

# Import constants so its easier to refer to column names
from markets_insights.core.column_definition import BaseColumns, CalculatedColumns

# get names of fwd looking price column names. Since, these column names are auto-generated there no constants for them
fwd_looking_price_fall_cols, fwd_looking_price_rise_cols = [x for x in daily_data.columns if 'TroughPerc' in x], \
    [x for x in daily_data.columns if 'PeakPerc' in x]

c:\Data\_Personal\Projects\markets_insights\src\markets_insights\calculations\base.py:386: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = data.groupby(self.get_group_cols(data.columns), group_keys=True).apply(


RsiCalculationWorker took 10 seconds
ColumnValueCrossedAboveFlagWorker took 1 seconds
ColumnValueCrossedBelowFlagWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds


In [22]:
daily_data[
(daily_data[CalculatedColumns.RsiCrossedAbove])
][fwd_looking_price_fall_cols].median()

TroughPercInNext1Sessions     1.417302
TroughPercInNext7Sessions     3.770465
TroughPercInNext15Sessions    4.783065
TroughPercInNext30Sessions    6.070147
TroughPercInNext45Sessions    6.783886
dtype: float64

In [23]:
daily_data[
(daily_data[CalculatedColumns.RsiCrossedBelow])
][fwd_looking_price_rise_cols].median()

PeakPercInNext1Sessions      3.876428
PeakPercInNext7Sessions      7.600800
PeakPercInNext15Sessions     9.780681
PeakPercInNext30Sessions    13.269178
PeakPercInNext45Sessions    16.275638
dtype: float64

### Performing Arithmetic on Readers

In [10]:
from markets_insights.datareader import data_reader
from markets_insights.core.core import IdentifierFilter

date_criteria = data_reader.DateRangeCriteria(datetime.date(2023, 1, 1), datetime.date(2023, 12, 31))

indices_reader = data_reader.NseIndicesReader()
vix_reader = data_reader.NseIndicesReader().set_filter(IdentifierFilter("India VIX"))
op_reader = indices_reader / vix_reader
data = op_reader.read(date_criteria).query(str(IdentifierFilter("Nifty 50 / India VIX")))

In [12]:
print(data.head(3).to_markdown())

|     | index-Identifier   | index-Index Date   |   index-Open |   index-High |   index-Low |   index-Close |   index-Points Change |   index-Change(%) |   index-Volume |   index-Turnover |   index-P/E |   index-P/B |   index-Div Yield | Date                |   index-PreviousClose | India VIX-Identifier   | India VIX-Index Date   |   India VIX-Open |   India VIX-High |   India VIX-Low |   India VIX-Close |   India VIX-Points Change |   India VIX-Change(%) |   India VIX-Volume |   India VIX-Turnover | India VIX-P/E   | India VIX-P/B   | India VIX-Div Yield   |   India VIX-PreviousClose | Identifier           |   Close |    High |     Low |    Open |
|----:|:-------------------|:-------------------|-------------:|-------------:|------------:|--------------:|----------------------:|------------------:|---------------:|-----------------:|------------:|------------:|------------------:|:--------------------|----------------------:|:-----------------------|:-----------------------|----------

### Extended Data Reader for Nasdaq

In [19]:
import yfinance as yf
import pandas
from markets_insights.datareader.data_reader import DateRangeDataReader
from markets_insights.core.core import Instrumentation
from markets_insights.core.column_definition import BaseColumns

In [21]:
class NasdaqDataReader (DateRangeDataReader):
  def __init__(self, tickers: list = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'NVDA']):
    super().__init__(reader=None)
    self.tickers = tickers
    self.name = "NasdaqDataReader"

  @Instrumentation.trace(name="NasdaqDataReader.read")
  def read(self, criteria: DateRangeCriteria):
    df_list = list()
    for ticker in self.tickers:
        data = yf.download(ticker, group_by="Ticker", start=criteria.from_date, end=criteria.to_date)
        data['ticker'] = ticker
        df_list.append(data)

    # combine all dataframes into a single dataframe
    df = pandas.concat(df_list)

    final_data = df.reset_index().rename(columns = self.get_column_name_mappings())
    final_data[BaseColumns.Date] = pandas.to_datetime(final_data[BaseColumns.Date])
    return final_data
  
  def get_column_name_mappings(self):
    return {
      'ticker': BaseColumns.Identifier,
      'OPEN': BaseColumns.Open,
      'HIGH': BaseColumns.High,
      'LOW': BaseColumns.Low,
      'CLOSE': BaseColumns.Close
    }

In [23]:
# import classes & setup options
import datetime
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder, HistoricalDataProcessOptions, DateRangeCriteria
from markets_insights.calculations.base import DatePartsCalculationWorker

reader = NasdaqDataReader()
options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(reader, DateRangeCriteria(year_start, to_date))

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('date_parts', CalculationPipelineBuilder.create_pipeline_for_worker(DatePartsCalculationWorker()))
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline
histDataProcessor.run_calculation_pipelines()

Started to read data
Reading data from 2023-01-02 to 2023-12-29


Failed to get ticker 'AAPL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: Exception('%ticker%: No timezone found, symbol may be delisted')
Failed to get ticker 'GOOGL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['GOOGL']: Exception('%ticker%: No timezone found, symbol may be delisted')
Failed to get ticker 'MSFT' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caus

NasdaqDataReader.read took 3 seconds
Saving data to file: ../../_data/processed/historical/NasdaqDataReader.csv
HistoricalDataProcessor.get_data took 3 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 0 seconds
HistoricalDataProcessor.process took 3 seconds
DatePartsCalculationWorker took 0 seconds
RsiCalculationWorker took 0 seconds


KeyError: 'Column not found: Rsi'

In [9]:
result.get_daily_data() \
  .sort_values(
    [BaseColumns.Date, BaseColumns.Identifier]
  )[
    [BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, 
     CalculatedColumns.RelativeStrengthIndex]
  ] \
  .tail(5)

KeyError: "['PreviousClose', 'Turnover (Rs. Cr.)', 'TOTTRDQTY'] not in index"

### Create a Calculation Pipeline

In [24]:
## import modules
from markets_insights.calculations.base import CalculationWorker
from markets_insights.core.core import Instrumentation
from markets_insights.calculations.base import BaseColumns
import pandas

# Implement the worker class. The important aspect here is to override the add_calculated_columns() method
class FibonacciRetracementCalculationWorker (CalculationWorker):
  def __init__(self, time_window: int, level_perct: float):
    self._time_window = time_window
    self._level = level_perct / 100
    self._column_name = 'Fbr' + str(level_perct)

  @Instrumentation.trace(name="FibnocciRetracementCalculationWorker")
  def add_calculated_columns(self, data: pandas.DataFrame):
    identifier_grouped_data: pandas.DataFrame = data.groupby(BaseColumns.Identifier)
    #Since, our dataframe may contain data for multiple symbols, we need to first group them by Identifier
    data[self._column_name] = identifier_grouped_data[BaseColumns.Close].transform(
        lambda x: 
          x.rolling(self._time_window).max() - 
          (
            (x.rolling(self._time_window).max() - x.rolling(self._time_window).min())  * self._level
          )
      )

In [25]:
# Create pipline with the FibnocciRetracementCalculationWorker and run 
from markets_insights.datareader.data_reader import NseIndicesReader, DateRangeCriteria
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, HistoricalDataProcessOptions, \
  MultiDataCalculationPipelines, CalculationPipeline
histDataProcessor = HistoricalDataProcessor(HistoricalDataProcessOptions(include_monthly_data=False, include_annual_data=False))

# Fetch the data
result = histDataProcessor.process(NseIndicesReader(), DateRangeCriteria(datetime.date(2023, 12, 1), datetime.date(2023, 12, 31)))

# Prepare calculation pipeline
fbr50_worker = FibonacciRetracementCalculationWorker(time_window=7, level_perct=50)
pipelines = MultiDataCalculationPipelines()
histDataProcessor.set_calculation_pipelines(
  CalculationPipeline(
    workers = [fbr50_worker]
  )
)

# Run the pipeline and get data
histDataProcessor.run_calculation_pipelines()

Started to read data
HistoricalDataProcessor.get_data took 0 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 0 seconds
HistoricalDataProcessor.process took 1 seconds
FibnocciRetracementCalculationWorker took 0 seconds


c:\Data\_Personal\Projects\markets_insights\src\markets_insights\dataprocess\data_processor.py:329: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([historical_data, manual_data], ignore_index=True)


In [26]:
## Display the results
print(result.get_daily_data()[[
  BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, fbr50_worker._column_name
]].tail(5).to_markdown())

|      | Identifier                  | Date                |    Close |    Fbr50 |
|-----:|:----------------------------|:--------------------|---------:|---------:|
| 2141 | NIFTY50 USD                 | 2023-12-29 00:00:00 |  9048.9  |  8941.77 |
| 2142 | NIFTY 10 YR BENCHMARK G-SEC | 2023-12-29 00:00:00 |  2232.79 |  2230.24 |
| 2143 | NIFTY200 ALPHA 30           | 2023-12-29 00:00:00 | 20858.6  | 20358.2  |
| 2144 | NIFTY HEALTHCARE INDEX      | 2023-12-29 00:00:00 | 10637.8  | 10422.7  |
| 2145 | NIFTY AUTO                  | 2023-12-29 00:00:00 | 18618.2  | 18070.9  |


## Preload Data

In [4]:
import datetime
from markets_insights.datareader.data_reader import DateRangeCriteria
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, HistoricalDataProcessOptions

options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

In [19]:
from markets_insights.datareader.data_reader import BhavCopyReader, NseIndicesReader
from datetime import date

result = histDataProcessor.process(NseIndicesReader(), DateRangeCriteria(date(1990, 1, 1), date(2024, 2, 28)))

Started to read data
Reading data from 2024-01-01 to 2024-02-28
HistoricalDataProcessor.sanitize_data took 3 seconds
Saving data to file: ../../_data/processed/historical/nse_equities.csv
HistoricalDataProcessor.get_data took 10 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.process took 10 seconds


In [8]:
try:
  10/0
except Exception as ex:
  print(ex)

division by zero


In [ ]:
result = histDataProcessor.process(BhavCopyReader(), DateRangeCriteria(date(2016, 1, 1), date(2024, 2, 28)))

In [13]:
import pandas as pd 
csv_data = pd.read_csv('../../_data/processed/historical/Indices.csv')

In [15]:
csv_data['Date'].str

In [9]:
eq_data['Date'] = pd.to_datetime(eq_data['Date'].str.replace(' 00:00:00', ''), format="%Y-%m-%d")